# Watermark Embedding and Extraction 

This notebook covers the embedding and extraction of watermarks in images using Singular Value Decomposition (SVD). It includes loading images, resizing, converting to grayscale, performing SVD operations, embedding the watermark, and extracting it.

## Embedding Watermark


This section describes the process of embedding a watermark into a host image using Singular Value Decomposition (SVD). We will go through the steps of loading the images, resizing and converting them to grayscale, performing SVD, embedding the watermark, and saving the watermarked image.

In [1]:
from PIL import Image
import numpy as np
import cv2
from tqdm import tqdm

### Load the host and watermark images

We start by loading the host image and the watermark image. These images will be used for the embedding process.

In [2]:
hr_image =  Image.open("./images/inputs/host_image.jpg")
wm_image =  Image.open("./images/inputs/watermark_image.jpg")

### Resize and Convert Images to Grayscale

Resize the watermark image to match the host image size and convert both images to grayscale.

In [3]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)

# Convert both images to grayscale
hr_image = hr_image.convert("L")
wm_image = wm_image_resized.convert("L")

### Convert Images to Numpy Arrays

Convert the grayscale images to numpy arrays for processing.

In [4]:
# Convert channels to numpy arrays
hr_image = np.array(hr_image)
wm_image = np.array(wm_image)
# hr_image,wm_image

### Function to Apply SVD and Modify Singular Values


This function applies Singular Value Decomposition (SVD) to both the host and watermark images, modifies the singular values of the host image by adding a scaled version of the watermark's singular values, and reconstructs the host image with the embedded watermark.

In [5]:
def embed_watermark(hr_channel, wm_channel, alpha=0.6):
    with tqdm(total=100, desc="Embedding Watermark", bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
        U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel)
        pbar.update(40)

        U_wm, sigma_wm, V_wm = np.linalg.svd(wm_channel)
        pbar.update(40)

        sigma_hr_mod = sigma_hr + alpha * sigma_wm
        hr_channel_mod = np.dot(U_hr[:, :sigma_hr_mod.shape[0]], np.dot(
            np.diag(sigma_hr_mod), V_hr[:sigma_hr_mod.shape[0], :]))
        pbar.update(20)

    return hr_channel_mod

### Embed Watermark


Embed the watermark into the host image by calling the `embed_watermark` function and save the watermarked image.

In [6]:
hr_image_mod = embed_watermark(hr_image, wm_image)

Embedding Watermark: 100%|██████████ [ time left: 00:00 ]


### Normalize and Convert to uint8

Normalize the modified image and convert it to an 8-bit unsigned integer format for proper display and storage.

In [7]:
# Normalize and convert to uint8
hr_image_mod = cv2.normalize(hr_image_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
# hr_image_mod

### Display and Save the Watermarked Image

Display the watermarked image and save it as `watermarked_image_grayscale.png`.

In [8]:
watermarked_image_grayscale = Image.fromarray(hr_image_mod)
watermarked_image_grayscale.show()
watermarked_image_grayscale.save('./images/outputs/watermarked_image_grayscale.png')

## Extracting Watermark


The watermark is extracted from the watermarked image by reversing the embedding process.

In [9]:
from PIL import Image
import numpy as np
import cv2

### Load the Host, Watermark, and Watermarked Images

Load the original host image, the original watermark image, and the watermarked image from which the watermark will be extracted.

In [10]:
hr_image = Image.open("./images/inputs/host_image.jpg")
wm_image = Image.open("./images/inputs/watermark_image.jpg")
watermarked_image = Image.open("./images/outputs/watermarked_image_grayscale.png")

### Resize and Convert Images to Grayscale

Resize the watermark and watermarked images to match the host image size, and convert all images to grayscale for extraction.


In [11]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)
watermarked_image_resized = watermarked_image.resize(hr_image.size)

# Convert images to RGB (if not already in that mode)
hr_image = hr_image.convert("L")
wm_image_resized = wm_image_resized.convert("L")
watermarked_image_resized = watermarked_image_resized.convert("L")

### Convert Images to Numpy Arrays

Convert the images to numpy arrays for processing during watermark extraction.

In [12]:
# Convert channels to numpy arrays
hr_image = np.array(hr_image)
wm_image = np.array(wm_image_resized)
watermarked_image = np.array(watermarked_image_resized)
# hr_image,wm_image,watermarked_image

### Extraction Process

Perform SVD on the watermarked image and the host image, calculate the difference in singular values, and reconstruct the watermark.

In [13]:
def extract_watermark(hr_mod_channel, hr_channel, wr_channel, alpha=0.6):
    with tqdm(total=100, desc="Extracting Watermark", bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
        U_hr_mod, sigma_hr_mod, V_hr_mod = np.linalg.svd(hr_mod_channel, full_matrices=False)
        pbar.update(30)

        U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel, full_matrices=False)
        pbar.update(30)

        U_wm, sigma_wm, V_wm = np.linalg.svd(wr_channel, full_matrices=False)
        pbar.update(30)

        sigma_wm_mod = (sigma_hr_mod - sigma_hr) / alpha
        wm_channel_mod = np.dot(U_wm, np.dot(np.diag(sigma_wm_mod), V_wm))
        pbar.update(10)

    return wm_channel_mod

### Extract Watermark

Extract the watermark from the watermarked image by calling the `extract_watermark` function and save the result.

In [14]:
wm_extracted = extract_watermark(watermarked_image, hr_image, wm_image)

Extracting Watermark: 100%|██████████ [ time left: 00:00 ]


### Normalize and Convert to uint8

Normalize the extracted watermark and convert it to an 8-bit unsigned integer format for display and storage.

In [15]:
# Normalize and convert to uint8
wm_extracted = cv2.normalize(wm_extracted, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
# wm_extracted

### Display and Save the extracted Watermark Image

Display the extracted watermark image and save it as `extracted_watermark_image_grascale.png`.

In [16]:
# Merge extracted channels back into an RGB image
extracted_watermark =Image.fromarray(wm_extracted)
extracted_watermark.save('./images/outputs/extracted_watermark_image_grascale.png')
extracted_watermark.show()